# Sumo Energy: Repeat Customers

In [2]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go

In [3]:
import pandas_gbq

## Loading data

In [5]:
query_all=''' SELECT
  ak_id, email, phone, display_name,
   upload_date, current_balance
 FROM
   books_db.books
 WHERE
   book_id in 
(select distinct book_id from 
presentation.dim_books where lower(Counterparty) like '%sumo energy%') '''

In [6]:
all = pandas_gbq.read_gbq(query_all,project_id="data-team-318806")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=VDBoxGCXhDSwKjx31cW38BP8TbQUnl&prompt=consent&access_type=offline
Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


In [7]:
query_repeat='''SELECT
  ak_id, email, phone, display_name,
  upload_date, current_balance
FROM
  books_db.books
WHERE
book_id in (select distinct book_id from 
presentation.dim_books where lower(Counterparty) like '%sumo energy%')
  AND
  (
  (email IS NOT NULL 
	 AND
	 email IN (
	    SELECT email
	    FROM books_db.books
      WHERE email IS NOT NULL AND email != 'nan' 
      AND book_id in (select distinct book_id from 
presentation.dim_books where lower(Counterparty) like '%sumo energy%')
	    GROUP BY email, last_name
	    HAVING COUNT(*) > 1
	  ))
  OR
  (
  phone IS NOT NULL
  AND
  phone IN (
    SELECT phone
        FROM books_db.books
    WHERE phone IS NOT NULL AND phone != 'nan' AND LENGTH(phone) >= 8 
    AND book_id in (select distinct book_id from 
presentation.dim_books where lower(Counterparty) like '%sumo energy%')
    GROUP BY phone, last_name
    HAVING COUNT(*) > 1
  ))
  )'''

In [8]:
repeat = pandas_gbq.read_gbq(query_repeat,project_id="data-team-318806")

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


In [9]:
# repeat[(repeat.email_count>2) |(repeat.phone_count>2)].head(30)

## Repeat accs

In [11]:
repeat.groupby('display_name')[['ak_id']].count()

,ak_id
display_name,
Sumo Energy AU Apr 2024 (Post DCA),88
Sumo Energy AU Apr 2024 (Pre DCA),26
Sumo Energy AU Aug 2024 (Pre DCA),8
Sumo Energy AU Dec 2023 (Post DCA),112
Sumo Energy AU Dec 2023 (Pre DCA),136
Sumo Energy AU Feb 2024 (Post DCA),31
Sumo Energy AU Feb 2024 (Pre DCA),49
Sumo Energy AU Jul 2024 (Post DCA),15
Sumo Energy AU Jul 2024 (Pre DCA),19


In [12]:
all.groupby('display_name')[['ak_id']].count()

,ak_id
display_name,
Sumo Energy AU Apr 2024 (Post DCA),220
Sumo Energy AU Apr 2024 (Pre DCA),92
Sumo Energy AU Aug 2024 (Pre DCA),60
Sumo Energy AU Dec 2023 (Post DCA),330
Sumo Energy AU Dec 2023 (Pre DCA),484
Sumo Energy AU Feb 2024 (Post DCA),120
Sumo Energy AU Feb 2024 (Pre DCA),146
Sumo Energy AU Jul 2024 (Post DCA),49
Sumo Energy AU Jul 2024 (Pre DCA),86


In [13]:
repeat['upload_date'] = pd.to_datetime(repeat['upload_date'], errors='coerce')


In [14]:
# adding first upload date

first_upl = (repeat.groupby(['phone', 'email'])
             ['upload_date'].min()
             .to_frame(name='first_upload_date')
            )
             
rows_before = repeat.shape[0]
assert repeat.join(first_upl, on=[ 'phone', 'email'], how='left').shape[0] == rows_before
repeat = repeat.join(first_upl, on=[ 'phone', 'email'], how='left')

In [15]:
repeat['first_upload_date'] = pd.to_datetime(repeat['first_upload_date'], errors='coerce')

In [16]:
#Arthur
repeat['first_upload_date'] = repeat['first_upload_date'].fillna(repeat['upload_date'])

In [17]:
# cohort lookup: old

cohort_dict = (repeat[['upload_date', 'display_name']]
               .drop_duplicates(subset=['upload_date'])  # Ensure unique upload_date
               .set_index('upload_date')
               .to_dict(orient='index')
              )
repeat['cohort'] = repeat['first_upload_date'].map(lambda x: cohort_dict[x]['display_name'])

In [18]:
# split to customers in their own cohort and 2nd/3rd appearances

own_cohort_df, next_df = repeat[repeat['upload_date'] == repeat['first_upload_date']], \
repeat[repeat['upload_date'] != repeat['first_upload_date']]

In [19]:
# first occurrence by phone
first_inst_phone = (repeat
                    .sort_values(by='upload_date')
                    .groupby(['phone'])['ak_id'].first().values
                   )

# first occurrence by email
first_inst_email = (repeat
                    .sort_values(by='upload_date')
                    .groupby(['email'])['ak_id'].first().values
                   )

In [20]:
# marking first occurrence in the initial file

repeat['fist_instance'] = 0
repeat.loc[repeat['ak_id'].isin(first_inst_phone), 'fist_instance'] = 1
repeat.loc[repeat['ak_id'].isin(first_inst_email), 'fist_instance'] = 1

repeat['fist_instance'].value_counts()

fist_instance
0    467
1    456
Name: count, dtype: int64

In [21]:
# Group by 'email' and 'display_name' and get the counts
email_counts = repeat.groupby(['email', 'display_name']).size().reset_index(name='email_count')
phone_counts = repeat.groupby(['phone', 'display_name']).size().reset_index(name='phone_count')

# Merge the counts back into the original DataFrame
repeat = repeat.merge(email_counts, on=['email', 'display_name'], how='left')
repeat = repeat.merge(phone_counts, on=['phone', 'display_name'], how='left')

In [22]:
# Condition for first_instance == 0
condition_0 = (repeat['fist_instance'] == 0)

# Condition for first_instance == 1 and counts > 1
condition_1 = (
    (repeat['fist_instance'] == 1) &
    ((repeat['email_count'] > 1) | (repeat['phone_count'] > 1))
)

# Combine the conditions
combined_condition = condition_0 | condition_1

# 
# Filter the DataFrame based on the combined condition
df_repeat = repeat[combined_condition]
df_repeat.shape

(678, 11)

### Join payment data by ak id

In [24]:
query_pay_rep='''SELECT
  display_name, payment_date, amount, ak_id
FROM
books_db.account_level_payments
WHERE
  ak_id IN (
    SELECT
      ak_id
    FROM
    books_db.books
    WHERE
       book_id in (select distinct book_id from 
presentation.dim_books where lower(Counterparty) like '%sumo energy%')
      AND (
        (email IS NOT NULL
        AND
          email IN (
            SELECT email
            FROM  books_db.books
            WHERE email IS NOT NULL AND email != 'nan' 
            AND  book_id in (select distinct book_id from 
presentation.dim_books where lower(Counterparty) like '%sumo energy%')
            GROUP BY email, last_name
            HAVING COUNT(*) > 1
          ))
        OR
        (
        phone IS NOT NULL
        AND
        phone IN (
          SELECT phone
          FROM  books_db.books
          WHERE phone IS NOT NULL AND phone != 'nan' AND LENGTH(phone) >= 8 
          AND  book_id in (select distinct book_id from 
presentation.dim_books where lower(Counterparty) like '%sumo energy%')
          GROUP BY phone, last_name
          HAVING COUNT(*) > 1
        ))
      )
  ) '''

In [25]:
 pay_rep=pandas_gbq.read_gbq(query_pay_rep,project_id="data-team-318806")

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


In [26]:
pay_rep.head()

,display_name,payment_date,amount,ak_id
0,Sumo Energy AU Apr 2024 (Pre DCA),2024-06-24 00:00:00+00:00,25.0,acee6c43-2683-4f74-877d-6ab31c46f354
1,Sumo Energy AU Apr 2024 (Pre DCA),2024-06-07 00:00:00+00:00,25.0,acee6c43-2683-4f74-877d-6ab31c46f354
2,Sumo Energy AU Apr 2024 (Pre DCA),2024-05-31 00:00:00+00:00,100.0,2576d1a0-06bf-4f12-a33c-12db13b780c7
3,Sumo Energy AU Apr 2024 (Pre DCA),2024-06-12 00:00:00+00:00,0.0,7a45c0cc-78fb-4c28-9513-90117666e5de
4,Sumo Energy AU Apr 2024 (Pre DCA),2024-06-05 00:00:00+00:00,0.0,7a45c0cc-78fb-4c28-9513-90117666e5de


In [27]:
# join payment data by ak id
# pay_rep = pd.read_csv('pay_rep.csv', parse_dates=['payment_date'])

pay_rep = pay_rep.groupby('ak_id')[['amount']].sum()

# Perform left merge
df_repeat = df_repeat.merge(pay_rep, on='ak_id', how='left')

# Fill NaN values in 'amount' with 0
df_repeat['amount'] = df_repeat['amount'].fillna(0)

In [28]:
# all repeat customers (without first instances)
df_agg = (df_repeat
        .groupby(['display_name'])
        .agg(
        total_repeat_accs=('ak_id', 'count'),
        # repeat_customers=('phone', 'nunique'),
        total_amount=('current_balance', 'sum'),
        average_amount=('current_balance', 'mean'),
        total_recoveries=('amount', 'sum'),
        total_payers=('amount', lambda x: (x != 0).sum()),
        upload_date=('upload_date', 'min'),
        )
        .round({'total_recoveries': 1, 'average_amount': 1, 'total_amount': 1})
        .sort_values(by='upload_date')
        .reset_index()
)

df_agg

,display_name,total_repeat_accs,total_amount,average_amount,total_recoveries,total_payers,upload_date
0,Sumo Energy AU Dec 2023 (Post DCA),93,43835.5,471.3,1266.4,6,2024-01-29 00:00:00+00:00
1,Sumo Energy AU Dec 2023 (Pre DCA),64,135336.2,2114.6,12842.8,7,2024-01-29 00:00:00+00:00
2,Sumo Energy AU Feb 2024 (Post DCA),18,4095.6,227.5,57.0,1,2024-02-28 00:00:00+00:00
3,Sumo Energy AU Feb 2024 (Pre DCA),33,53018.8,1606.6,368.9,3,2024-02-28 00:00:00+00:00
4,Sumo Energy AU Mar 2024 (Pre DCA),5,7445.8,1489.2,0.0,0,2024-03-25 00:00:00+00:00
5,Sumo Energy AU Mar 2024 (Post DCA),98,35025.0,357.4,1976.0,10,2024-03-25 00:00:00+00:00
6,Sumo Energy AU Apr 2024 (Post DCA),76,26959.9,354.7,1252.9,6,2024-04-28 00:00:00+00:00
7,Sumo Energy AU Apr 2024 (Pre DCA),14,22933.8,1638.1,100.0,2,2024-04-28 00:00:00+00:00
8,Sumo Energy AU May 2024 (Pre DCA),15,29625.0,1975.0,1369.1,2,2024-05-28 00:00:00+00:00
9,Sumo Energy AU May 2024 (Post DCA),56,17210.0,307.3,275.5,2,2024-05-28 00:00:00+00:00


In [29]:
# Aggregate metrics into a two-column format
total_repeat_accs = df_repeat['ak_id'].count()
total_amount = df_repeat['current_balance'].sum()
average_amount = df_repeat['current_balance'].mean()
total_recoveries= df_repeat['amount'].sum()
customers_amount = df_repeat['email'].nunique()
total_payers = (df_repeat['amount'] != 0).sum()

# Format results as integers
df_agg_total = pd.DataFrame({
    'name': ['total_repeat_accs', 'total_amount', 'average_amount', 'customers_amount','total_recoveries','total_payers'],
    'val': [int(total_repeat_accs), int(total_amount), int(average_amount), int(customers_amount), int(total_recoveries), int(total_payers)],
})

df_agg_total

,name,val
0,total_repeat_accs,678
1,total_amount,522144
2,average_amount,770
3,customers_amount,427
4,total_recoveries,20085
5,total_payers,42


In [30]:
###old
# Function to transform the date string
def transform_date_string(date_str):
    # Split the string and extract the month and year
    parts = date_str.split()
    month = parts[2]
    year = parts[3][2:]  # Take the last two digits of the year
    return f"{month} {year}"

# Apply the transformation to the 'cohort' column
df_repeat['cohort_tableau'] = df_repeat['cohort'].apply(transform_date_string)

In [34]:
df_repeat.to_csv('tableau_repeat.csv', index=False)

In [36]:
query_pay_all='''SELECT
display_name, payment_date, amount, ak_id
FROM
books_db.account_level_payments
WHERE
ak_id IN (
SELECT
ak_id
FROM
books_db.books
WHERE
book_id in (select distinct book_id from 
presentation.dim_books where lower(Counterparty) like '%sumo energy%'))'''

In [37]:
 pay_all=pandas_gbq.read_gbq(query_pay_all,project_id="data-team-318806")

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████|


## All customers

In [39]:
# join payment data by ak id
# pay_all = pd.read_csv('pay_all.csv', parse_dates=['payment_date'])

pay_all = pay_all.groupby('ak_id')[['amount']].sum()

# Perform left merge
all = all.merge(pay_all, on='ak_id', how='left')

# Fill NaN values in 'amount' with 0
all['amount'] = all['amount'].fillna(0)

In [40]:
all['upload_date'] = all['upload_date'].astype(str)
all['upload_date'] = pd.to_datetime(all['upload_date'].str.split(' ').str[0])


# all repeat customers (without first instances)
all_agg = (all
        .groupby(['display_name'])
        .agg(
        total_accs=('ak_id', 'count'),
        total_amount=('current_balance', 'sum'),
        average_amount=('current_balance', 'mean'),
        total_recoveries=('amount', 'sum'),
        total_payers=('amount', lambda x: (x != 0).sum()),
        upload_date=('upload_date', 'min'),
        )
        .round({'total_recoveries': 1, 'average_amount': 1, 'total_amount': 1})
        .sort_values(by='upload_date')
        .reset_index()
)

all_agg

,display_name,total_accs,total_amount,average_amount,total_recoveries,total_payers,upload_date
0,Sumo Energy AU Dec 2023 (Post DCA),330,162216.1,491.6,6990.7,26,2024-01-29
1,Sumo Energy AU Dec 2023 (Pre DCA),484,980604.6,2026.0,58591.1,54,2024-01-29
2,Sumo Energy AU Feb 2024 (Post DCA),120,38340.4,319.5,1687.4,12,2024-02-28
3,Sumo Energy AU Feb 2024 (Pre DCA),146,271139.8,1857.1,14594.0,17,2024-02-28
4,Sumo Energy AU Mar 2024 (Pre DCA),79,152018.9,1924.3,7175.7,8,2024-03-25
5,Sumo Energy AU Mar 2024 (Post DCA),294,101573.6,345.5,6086.3,31,2024-03-25
6,Sumo Energy AU Apr 2024 (Post DCA),220,78780.8,358.1,5504.8,24,2024-04-28
7,Sumo Energy AU Apr 2024 (Pre DCA),92,164091.5,1783.6,6913.6,11,2024-04-28
8,Sumo Energy AU May 2024 (Pre DCA),147,258270.1,1756.9,14910.2,20,2024-05-28
9,Sumo Energy AU May 2024 (Post DCA),188,63786.1,339.3,3851.2,17,2024-05-28


In [41]:
# Aggregate metrics into a two-column format
total_accs = all['ak_id'].count()
total_amount = all['current_balance'].sum()
average_amount = all['current_balance'].mean()
total_recoveries= all['amount'].sum()
customers_amount = all['email'].nunique()
total_payers = (all['amount'] != 0).sum()

# Format results as integers
df_agg_all = pd.DataFrame({
    'name': ['total_accs', 'total_amount', 'average_amount', 'customers_amount','total_recoveries','total_payers'],
    'val': [int(total_accs), int(total_amount), int(average_amount), int(customers_amount), int(total_recoveries), int(total_payers)],
})

df_agg_all

,name,val
0,total_accs,2927
1,total_amount,3109899
2,average_amount,1062
3,customers_amount,2373
4,total_recoveries,151427
5,total_payers,253


In [42]:
all_agg.to_csv('all_agg.csv')

In [43]:
df_agg.to_csv('df_agg.csv')